Check the action space and observation space of the environment

In [2]:
from HohmannTransferEnv import HohmannTransferEnv

env = HohmannTransferEnv()
print(env.action_space)
print(env.observation_space)

Box(-0.1, 0.1, (2,), float32)
Box(-inf, inf, (5,), float32)
